In [0]:
from etl.constants import *
from etl.data_extractor import *
from etl.data_standardizer import *

In [0]:

extractor = DataExtractor(spark)

raw_dataframes = extractor.extract()

standardizer = DataStandardizer(raw_dataframes)
cleaned_dataframes = standardizer.process()